<a href="https://colab.research.google.com/github/elivasquezhdz/3dgan-release/blob/master/Invoice_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install non native dependencies

In [0]:
!pip install pyramid
!pip install Flask-SQLAlchemy
!pip install -U -q PyDrive

#Create db model

In [0]:
#This will donload the file invoice.py from google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = "1_VYzV_O8gJaqQHXvtO7edKky3U966Hk3"


downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('invoice.py')



In [0]:
#Once the invoice script will create the model
from invoice import db
db.create_all()

#Make the app public with NGROK

In [8]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-02-04 08:18:23--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.206.253.53, 34.199.255.1, 52.204.188.97, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.206.253.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  --.-KB/s    in 0.1s    

2019-02-04 08:18:23 (46.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [10]:
#click on link after running pyramid app
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://635ca9d3.ngrok.io


#Run Pyramid app

In [0]:
#Pyramid hellow world on port 6006
from wsgiref.simple_server import make_server
from pyramid.config import Configurator
from pyramid.response import Response


def hello_world(request):
    return Response('Hello World!')


if __name__ == '__main__':
    with Configurator() as config:
        config.add_route('hello', '/')
        config.add_view(hello_world, route_name='hello')
        app = config.make_wsgi_app()
    server = make_server('0.0.0.0', 6006, app)
    server.serve_forever()